In [1]:
# Imports

import warnings

warnings.filterwarnings("ignore")

# Import libraries
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil import relativedelta
import operator
import os
import random
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
import matplotlib.ticker as ticker
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: f'%.{len(str(x%1))-2}f' % x)
pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [2]:
trans = pd.read_csv('CS_TRANS_DETAIL_202305161206.csv',converters={'CIF_ID':str})
detail = pd.read_csv('CS_DETAIL_202305161209.csv',converters={'CIF_ID':str})
eod = pd.read_csv('CS_EOD_BAL_202305161211.csv',converters={'CIF_ID':str})

In [3]:
trans.sample(10)

,CIF_ID,PART_TRAN_TYPE,TRAN_CRNCY_CODE,TRAN_DATE,TRAN_AMT
1491809,0269475,D,KES,2022-12-30 00:00:00.000,422.5600000000000023
585642,0311585,D,KES,2022-04-07 00:00:00.000,3000.0
704049,0321129,C,KES,2022-05-20 00:00:00.000,250.0
727692,0200285,D,KES,2022-05-28 00:00:00.000,33940.0
646270,0053272,D,KES,2022-04-30 00:00:00.000,250.0
235757,0329365,D,KES,2021-11-04 00:00:00.000,2000.0
35629,0037195,D,KES,2021-07-21 00:00:00.000,3798.0
1045681,0334297,D,KES,2022-08-31 00:00:00.000,200.0
948657,0310287,D,KES,2022-08-01 00:00:00.000,15000.0
1094161,0351834,D,KES,2022-09-14 00:00:00.000,4000.0


In [4]:
trans['PART_TRAN_TYPE'].unique()

array(['D', 'C'], dtype=object)

In [5]:
trans_cr = trans[trans['PART_TRAN_TYPE'] == 'C']
trans_dr = trans[trans['PART_TRAN_TYPE'] == 'D']

In [6]:
trans_cr = pd.merge(trans_cr,detail[['CIF_ID','DISB_DATE_6','APPLICABLE_DATE']],
                   how='left',on='CIF_ID')
trans_dr = pd.merge(trans_dr,detail[['CIF_ID','DISB_DATE_6','APPLICABLE_DATE']],
                   how='left',on='CIF_ID')

In [7]:
trans_cr.sample(3)

,CIF_ID,PART_TRAN_TYPE,TRAN_CRNCY_CODE,TRAN_DATE,TRAN_AMT,DISB_DATE_6,APPLICABLE_DATE
193499,0075993,C,KES,2022-04-26 00:00:00.000,79967.0,2021-12-20 00:00:00.000,2022-06-20 00:00:00.000
163478,0178729,C,KES,2022-03-26 00:00:00.000,30.0,2022-06-19 00:00:00.000,2022-12-19 00:00:00.000
448995,0092672,C,KES,2022-10-31 00:00:00.000,200.0,2022-03-02 00:00:00.000,2022-09-02 00:00:00.000


In [8]:
trans_cr['TRAN_DATE'] = pd.to_datetime(trans_cr['TRAN_DATE'])
trans_cr['DISB_DATE_6'] = pd.to_datetime(trans_cr['DISB_DATE_6'])
trans_cr['APPLICABLE_DATE'] = pd.to_datetime(trans_cr['APPLICABLE_DATE'])
trans_dr['DISB_DATE_6'] = pd.to_datetime(trans_dr['DISB_DATE_6'])
trans_dr['TRAN_DATE'] = pd.to_datetime(trans_dr['TRAN_DATE'])
trans_dr['APPLICABLE_DATE'] = pd.to_datetime(trans_dr['APPLICABLE_DATE'])

In [9]:
trans_cr_unique = trans_cr[['CIF_ID','DISB_DATE_6','APPLICABLE_DATE']]
trans_cr_unique.drop_duplicates(keep='first',inplace=True)
trans_dr_unique = trans_dr[['CIF_ID','DISB_DATE_6','APPLICABLE_DATE']]
trans_dr_unique.drop_duplicates(keep='first',inplace=True)

In [10]:
trans_cr_unique.shape, trans_dr_unique.shape

((2372, 3), (2372, 3))

In [11]:
def credit_aggregates(CIF_ID, DISB_DATE_6, APPLICABLE_DATE, df):
    aggr_df = df.copy()
    aggr_df = aggr_df[(aggr_df['CIF_ID'] == CIF_ID) & (aggr_df['TRAN_DATE'] >= DISB_DATE_6) & \
           (aggr_df['TRAN_DATE'] <= APPLICABLE_DATE)]
    return np.min(aggr_df['TRAN_AMT']), np.max(aggr_df['TRAN_AMT']), np.mean(aggr_df['TRAN_AMT']), np.median(aggr_df['TRAN_AMT'])

In [12]:
def debit_aggregates(CIF_ID, DISB_DATE_6, APPLICABLE_DATE, df):
    aggr_df = df.copy()
    aggr_df = aggr_df[(aggr_df['CIF_ID'] == CIF_ID) & (aggr_df['TRAN_DATE'] >= DISB_DATE_6) & \
           (aggr_df['TRAN_DATE'] <= APPLICABLE_DATE)]
    return np.min(aggr_df['TRAN_AMT']), np.max(aggr_df['TRAN_AMT']), np.mean(aggr_df['TRAN_AMT']), np.median(aggr_df['TRAN_AMT'])

In [15]:
def eod_aggregates(CIF_ID, df):
    aggr_df = df.copy()
    aggr_df = aggr_df[aggr_df['CIF_ID'] == CIF_ID]
    return np.mean(aggr_df['TRAN_DATE_BAL']), np.median(aggr_df['TRAN_DATE_BAL'])

In [14]:
trans_cr_unique['min_cred_amt_6_months'] = \
trans_cr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_cr)[0], axis=1)
trans_cr_unique['max_cred_amt_6_months'] = \
trans_cr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_cr)[1], axis=1)
trans_cr_unique['avg_cred_amt_6_months'] = \
trans_cr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_cr)[2], axis=1)
trans_cr_unique['median_cred_amt_6_months'] = \
trans_cr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_cr)[3], axis=1)

In [17]:
trans_dr_unique['min_cred_amt_6_months'] = \
trans_dr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_dr)[0], axis=1)
trans_dr_unique['max_cred_amt_6_months'] = \
trans_dr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_dr)[1], axis=1)
trans_dr_unique['avg_cred_amt_6_months'] = \
trans_dr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_dr)[2], axis=1)
trans_dr_unique['median_cred_amt_6_months'] = \
trans_dr_unique.apply(lambda x: credit_aggregates(x.CIF_ID, x.DISB_DATE_6, x.APPLICABLE_DATE, trans_dr)[3], axis=1)

In [18]:
trans_dr_unique.rename(columns={'min_cred_amt_6_months': 'min_debit_amt_6_months',
                               'max_cred_amt_6_months': 'max_debit_amt_6_months',
                               'avg_cred_amt_6_months': 'avg_debit_amt_6_months',
                               'median_cred_amt_6_months': 'median_debit_amt_6_months'}, inplace=True)

In [20]:
trans_dr_unique.shape

(2372, 7)

In [21]:
trans_cr_unique['avg_eod_bal_6_months'] = \
trans_cr_unique.apply(lambda x: eod_aggregates(x.CIF_ID, eod)[0], axis=1)
trans_cr_unique['median_eod_bal_6_months'] = \
trans_cr_unique.apply(lambda x: eod_aggregates(x.CIF_ID, eod)[1], axis=1)

In [23]:
del trans_cr_unique['DISB_DATE_6']
del trans_cr_unique['APPLICABLE_DATE']
del trans_dr_unique['DISB_DATE_6']
del trans_dr_unique['APPLICABLE_DATE']

In [24]:
final_df = pd.merge(trans_cr_unique, trans_dr_unique, how='left', on = 'CIF_ID')

In [26]:
final_df.sample(10)

,CIF_ID,min_cred_amt_6_months,max_cred_amt_6_months,avg_cred_amt_6_months,median_cred_amt_6_months,avg_eod_bal_6_months,median_eod_bal_6_months,min_debit_amt_6_months,max_debit_amt_6_months,avg_debit_amt_6_months,median_debit_amt_6_months
1281,0180041,60.0,469000.0,38567.46875,3000.0,15796.9828571428570285,0.64,1.0,150000.0,7209.7854545454547406,200.0
963,0042941,17442.0,1000000.0,138895.37037037036498077,89553.0,45338.4156521739132586,12053.37999999999919964,10.0,349282.27000000001862645,29709.7886813186814834,6800.0
1655,0285120,8.0,2000000.0,176709.9333333333197515,54.5,225322.049310344824334607,170325.5,10.0,250000.0,28353.5523140495861298,14600.0
610,0326101,6.5499999999999998,100000.0,14628.2775000000001455,5000.0,3250.9864814814809506,71.5750000000000028,0.62,96000.0,4977.7733333333335395,1000.0
110,0299218,10.0,70000.0,18595.06631578947417438,15000.0,2595.4468749999996362,118.2600000000000051,3.2599999999999998,40000.0,5104.1289855072463979,82.2600000000000051
2015,0341696,420.0,300000.0,43847.0,33477.0,5691.058823529411711206,568.0,500.0,150000.0,21922.0,4391.0
2116,0343735,2.0,132735.0,60459.4315384615401854,87928.0,32807.1900000000023283,18154.3600000000005821,6.0,100000.0,17735.5348717948727426,4000.0
1746,0019889,65776.0,1000000.0,199332.0,65776.0,47740.6827586206854903,31033.2000000000007276,1.0,500000.0,16427.21917808219222934,600.0
1312,0171664,1.0,650000.0,44969.2368421052597114,4000.0,17668.33720930232448154,862.5,70.0,150000.0,11661.4264705882360431,2500.0
1536,0043693,298.0,3000000.0,376565.6090909091290087,146610.39999999999417923,136969.8406818181974813,62058.45500000000174623,40.0,130000.0,36253.11111111110949423,30000.0


In [27]:
final_df['CIF_ID'] = final_df['CIF_ID'].apply('="{}"'.format)

In [28]:
final_df.to_csv('trans_eod.csv')